In [1]:
import numpy as np
import pandas as pd 
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier 
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support , accuracy_score
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
import sys
import lmdb
import h5py
caffe_root = '/work/sagarj/caffe-rc5/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')
from collections import defaultdict
import caffe


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
model_root = "/work/sagarj/Work/BellLabs/caffe_models/caffe_model_1/"
# net_weights='caffe_beauty_augmented/caffe_model_beauty_city_smartAugment_fixed_iter_20000.caffemodel'
# net_weights='caffe_model_beauty_4_votes/caffe_model_1_iter_24732.caffemodel'
# net_weights = '../caffe_models/caffe_model_1/smartAugment/caffe_model_beauty_city_smartAugment_iter_15000.caffemodel'
net_weights='caffe_model_beauty_city_smartAugment_fixed_iter_20000.caffemodel'
net_definition='caffenet_deploy_1.prototxt'

test_lmdb_path = '../Data/validation_lmdb_beauty_augmented_smart/' # Test LMDB database path
mean_file_binaryproto = '../Data/citySmart1.binaryproto' # Mean image file


In [4]:
caffe.set_mode_gpu()
model_def = model_root+net_definition
model_weights = model_root + net_weights

In [5]:
net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)


In [6]:
np.asarray(net.blobs['fc8'].data)

array([[0., 0.]], dtype=float32)

In [7]:
# Extract mean from the mean image file
mean_blobproto_new = caffe.proto.caffe_pb2.BlobProto()
f = open(mean_file_binaryproto, 'rb')
mean_blobproto_new.ParseFromString(f.read())
mean_image = caffe.io.blobproto_to_array(mean_blobproto_new)
f.close()

In [ ]:
count = 0
correct = 0
labels = [] # (real,pred) -> int
preds = []
# labels_set = set()

lmdb_env = lmdb.open(test_lmdb_path)
lmdb_txn = lmdb_env.begin()
lmdb_cursor = lmdb_txn.cursor()

for key, value in lmdb_cursor:
    datum = caffe.proto.caffe_pb2.Datum()
    datum.ParseFromString(value)
    label = datum.label
    labels.append(label)
    image = caffe.io.datum_to_array(datum)
    image = image.astype(np.uint8)
    out = net.forward_all(data=np.asarray([image]) - mean_image).copy()

    plabel = out['prob']
    preds.append(plabel)
    count += 1

In [ ]:
preds[3]

In [ ]:
pred_label = [np.argmax(i) for i in preds ]

In [ ]:
pred_label[2] , labels[2]

In [ ]:
error_RF = mean_squared_error(labels, pred_label)

precision , recall , fscore , _ = precision_recall_fscore_support(labels, pred_label)


print ( precision , recall , fscore , _)

In [ ]:
accuracy_score(labels,pred_label)

In [14]:
import scipy.stats as stats
tau , pvalue = stats.kendalltau(labels,pred_label)

In [15]:
print tau , pvalue

0.4030028460604292 1.383601688943125e-86
